# Übungen zu Teilchenphysik I
## Exercise 03 - EMCal in a nutshell

    D. Wong, November 2025                                                

## Setup

It is very likely that you will need the following packages, so don't forget to import them!

In [ ]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
# This is a local module that will be necessary for the sections 2 and 3: it's already provided in this repository
import exercise3_utils as ex3

<a name='section_1_0'></a>
<hr style="height: 1px;">


## <h1 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Section 1: Electromagnetic cascades in a calorimeter</h1>


<div style="text-align: center; font-family: 'Arial', sans-serif; font-size: 24px; line-height: 1.5; color: yellow; background-color: black; padding: 20px;">
    <img src="https://static1.cbrimages.com/wordpress/wp-content/uploads/2022/05/Darth-Varder-Lightning.jpg?q=50&fit=crop&w=1140&h=&dpr=1.5" alt="Darth Vader" width="300" style="float: left; margin-right: 10px;">

<p style="font-size: 34px; font-weight: bold;">Anakin mastered the dark side — and the bright sparks of E&M.</p>
</div>


An **electromagnetic (EM) shower** is a cascade of particles, including photons, electrons, and positrons, forms when a high-energy electron or photon interacts with dense material and produces a chain of secondary particles. Photons convert into electron–positron pairs, and these charged particles in turn emit bremsstrahlung photons, repeating the process as the shower multiplies and spreads. The cascade continues until particle energies fall below a critical value, where ionization dominates and the shower dies out. In an electromagnetic calorimeter (EMCal), this process is harnessed to measure particle energies: alternating layers of absorber and active material contain and sample the shower, converting the deposited energy into measurable signals. The total signal provides a precise estimate of the incident electron or photon’s energy and impact position, making EM calorimetry a key technique in modern high-energy physics experiments.

![EMShower](https://www.aanda.org/articles/aa/full/2003/43/aaINTEGRAL41/img17.gif)



<a name='section_1_1'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 1.1: EMCal dimension estimation</h3>

**Electromagnetic calorimeters (EMCal)** designed to measure EM showers use high-Z materials to trap the shower. The electrons in the shower produce scintillation light, and the amount of light collected is proportional to the total energy of the incident particles. This makes EMCal ideal for precisely measuring the energy of electrons, positrons and photons.

The CMS detector at the LHC uses lead tungstate (PbWO$_4$) as the EMCal material.

<div class="alert alert-info" role="alert">
  <strong>Exercise:</strong>
  <ul>
    <li>Compute the mixture radiation length \(X_0\) (in g/cm\(^2\)) from elemental \(X_0\) using mass-fraction weighting.</li>
    <li>Convert to a length (cm) using the material density.</li>
    <li>Use the definition \(E_c \approx \left(\frac{dE}{dx}\right)_{\text{coll}} \times X_0\), i.e., set collisional losses equal to radiative losses (with \(b \simeq 1/X_0\)).</li>
  </ul>
  <p>
    A reasonable collisional <em>mass</em> stopping power for electrons around ~10&nbsp;MeV in a high-\(Z\) crystal is about
    \(1.30\ \text{MeV}\,\text{cm}^2/\text{g}\). With standard PDG elemental \(X_0\) values and the density for
    PbWO\(_4\), this yields \(E_c \approx 9.64\ \text{MeV}\).
  </p>
  <p>
    The radiation length \(X_0\) sets the scale for EM shower development—on average, it is the distance over which an
    electron loses \(1/e\) of its energy by bremsstrahlung.
  </p>
</div>


In [ ]:
import numpy as np

# Stoichiometry: Pb W O4   -> 1 Pb, 1 W, 4 O
atomic_masses = {'Pb': 207.2, 'W': 183.84, 'O': 16.0}   # g/mol
stoich = {'Pb': 1, 'W': 1, 'O': 4}

# Elemental radiation lengths X0 (PDG) in g/cm^2
X0_elem_gcm2 = {'Pb': 6.37, 'W': 6.76, 'O': 34.24}

# Bulk density of PbWO4 (g/cm^3)
rho = 8.28

# Collisional MASS stopping power for ~10 MeV electrons in this medium (MeV cm^2 / g)
# 1.30 is a good default that reproduces ~9.6 MeV with standard PDG inputs.
S_coll_mass = 1.30

# ----- Helper: mass fractions from stoichiometry -----
M_molecule = sum(stoich[el] * atomic_masses[el] for el in stoich)
w_mass = {el: (stoich[el] * atomic_masses[el]) / M_molecule for el in stoich}

# ----- Mixture rule for radiation length: 1/X0 = sum(w_i / X0_i) -----
inv_X0_mix_gcm2 = sum(w_mass[el] / X0_elem_gcm2[el] for el in stoich)
X0_mix_gcm2 = 1.0 / inv_X0_mix_gcm2           # g/cm^2
X0_mix_cm   = X0_mix_gcm2 / rho               # cm

# ----- Critical energy: Ec ≈ (dE/dx)_coll × X0  -----
# (dE/dx)_coll in MeV/cm = (S_coll_mass in MeV cm^2/g) × (rho in g/cm^3)
S_coll_MeV_per_cm = S_coll_mass * rho
Ec_MeV = S_coll_MeV_per_cm * X0_mix_cm

print(f"PbWO4:")
print(f"  Radiation length X0 ≈ {X0_mix_gcm2:.3f} g/cm^2  ({X0_mix_cm*10:.2f} mm)")
print(f"  Using S_coll_mass = {S_coll_mass:.2f} MeV·cm^2/g")
print(f"  -> Electron critical energy Ec ≈ {Ec_MeV:.2f} MeV")


The effective atomic number and Thomson’s approximation cannot produce a good estimation for the radiation length and the critical energy.

Consuld the PDG website (https://pdg.lbl.gov/2024/AtomicNuclearProperties/) to get more precise values for both radiation length and critical energy for PbWO$_4$ (note that lead tungstate is an inorganic scintillator).

<div class="alert alert-info">
<strong>Exercise:</strong> 
Calculate the approximate dimension of PbWO$_4$ crystal (longitudinal depth and transverse width) for a 100 GeV electron </span>
</div>

In [ ]:
import math

# Beam & material (PbWO4)
E = 100_000.0       # MeV (100 GeV)
rho = 8.30          # g/cm^3
X0_gcm2 = 7.39      # g/cm^2  (PbWO4)
Ec = 9.64           # MeV
Es = 21.2           # MeV (scale energy)

X0_cm = X0_gcm2 / rho

# Molière radius and an illustrative width ~2*R_M
R_M = (Es / Ec) * X0_cm
width = 2.0 * R_M

print(f"Molière radius R_M ≈ {R_M:.2f} cm")
print(f'Width (~2 R_M)     ≈ {width:.2f} cm')

t_max = math.log(E / Ec) - 0.5
x_max_cm = t_max * X0_cm

print(f"t_max ≈ {t_max:.2f} X0")
print(f"x_max ≈ {x_max_cm:.2f} cm")


<div class="alert alert-info">
<strong>Exercise:</strong> 
Is this a good estimation for EMCal size? CMS EMCal crystals are actually 25 $X_0$ long—why?</span>
</div>

In [ ]:
# We want to measure the entire electromagnetic shower: to cover the whole shower we need redundancy, otherwise the shower
# "leaks" outside the crystal and we miss part of it.

<a name='section_1_2'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 1.2: Shape of muon clusters on EMCal</h3>

For bremsstrahlung process, the energy loss through distance is given by $-\frac{dE}{dx} \propto \frac{Z^2 E}{m_{particle}^2}$.

<div class="alert alert-info">
<strong>Exercise:</strong> 
Consider a muon with a momentum of 50 GeV. What is the shape of the shower on an EMCal? How do you expect the energy deposit to be distributed on an EMCal?</span>
</div>

In [ ]:
# A 50 GeV muon is still a MIP. We expect a point-like EMCal with very little energy deposit (narrow peak near 0 GeV)

<div class="alert alert-info">
<strong>Exercise:</strong> 
Knowing at what energy electrons and positrons start emitting significant bremsstrahlung (what energy?), determine the threshold energy for a muon to emit significant bremsstrahlung in a PbWO$_4$ EMCal.</span>
</div>

In [ ]:
# The energy at which electrons and positrons start emitting significant energy via bremsstrahlung is the critical energy.
# Let's consider PbWO4: we already got from PDG that the critical energy for an electron is 9.64 MeV in.
muon_mass = 105.658 # MeV
electron_mass = 0.510999 # MeV
ratio2 = (muon_mass / electron_mass)**2
E_c_electron = 9.64  # MeV in PbWO4 (PDG)
E_c_muon = ratio2 * E_c_electron
print(f"The critical energy for a muon in PbWO4 is ~{E_c_muon/1000:.0f} GeV")

<a name='section_1_3'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 1.3: Detector proposal</h3>

<div class="alert alert-info">
<strong>Exercise:</strong> 
How would you imeplement a discriminator with EMCAL data to recognize electrons from photons and muons?</span>
</div>

In [ ]:
# Electrons and muons can be distinguished via an electromagnetic calorimeter.
# Electrons and photons look identical in an EMCal: we need a charged particle tracker to find the tracks left by the electrons.

<a name='section_2_0'></a>
<hr style="height: 1px;">


## <h1 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Section 2: Calorimetry and reconstruction</h1>


The EMCal at the PHENIX experiment has in total, 2592 towers or channels to read out the energy deposits. The channels are arranged in a (72 x 36) matrix. For more details, please refer to the documentation.

Use the following code snippet to read one event from the EMCal simulation:

```
# Example to obtain EMCal hits
elmID, edep = ex3.get_hit_data()
edep = edep/ex3.sfc  # This converts the energy depositions into GeV
```

In the rest of the exercise, whenever you are asked to work with the simulated events from the PHENIX EMCal, please remember to always conver the energy depositions as above using `ex3.sfc`!

<a name='section_2_1'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 2.1: Events visualization and distribution</h3>

The `elmID` is the index of the channel that received an hit and `edep` is the energy deposition measured for a hit in the given channel. To reconstruct the energy of the particle, you need to convert `elmID` into 2D spatial coordinates. Also, the energy deposited in the EMCal should be divided by a sampling fraction constant (`ex3.sfc`).

<div class="alert alert-info">
<strong>Exercise:</strong> 
Write an algorithm that converts the channel ID into a pair of X and Y coordinates (in cm) according to the geometry of the PHENIX EMCal.</span>
</div>

In [ ]:
def map_channels_energies_to_matrix(channels, energies, n_rows=72, n_columns=36):
    # This function does not map into spatial coordinates: useful to cross-check the results
    shape = (n_rows, n_columns)
    matrix = np.zeros(shape, dtype=float)
    i_rows, i_columns = np.unravel_index(channels, shape)
    matrix[i_rows, i_columns] = energies
    return matrix.T


def channel_to_spatial_coordinates(channels, n_rows=72, n_columns=36, channel_size=5.535):
    shape = (n_rows, n_columns)
    i_rows, i_columns = np.unravel_index(channels, shape)
    x = (i_rows - n_rows / 2) * channel_size
    y = (n_columns / 2 - i_columns) * channel_size
    return np.column_stack((x, y))

<div class="alert alert-info">
<strong>Exercise:</strong> 
Plot the 2D distribution of all hits in an event according to their X and Y coordinates and their energy deposition. For example, show event 5 from the electron sample.</span>
</div>

In [ ]:
def plot_simple_energy_matrix(matrix, title="EMCal clusters", color_map="viridis"):
    import matplotlib  # noqa
    masked_matrix = np.ma.masked_where(matrix == 0, matrix)
    cmap = matplotlib.colormaps.get_cmap(color_map)
    cmap.set_bad(color='white')
    plt.figure(figsize=(10, 10))
    plt.imshow(masked_matrix, cmap=cmap, aspect='auto')
    plt.colorbar(label="Energy deposition (GeV)")
    plt.title(title)
    plt.xlabel("X coordinate (arbitrary units)")
    plt.ylabel("Y coordinate (arbitrary units)")
    plt.show()


def plot_spatial_energy_matrix(coordinates, energies, centroids=None, labels=None, title="EMCal clusters", marker_size_scale=20, color_map="viridis"):
    x, y = coordinates[:, 0], coordinates[:, 1]
    plt.figure(figsize=(10, 10))
    
    scatter = plt.scatter(
        x, y,
        c=energies,
        s=energies * marker_size_scale,
        cmap=color_map,
        edgecolor="k",
        alpha=0.8
    )
    
    plt.colorbar(scatter, label="Energy deposition (GeV)")
    plt.title(title)
    plt.xlabel("X coordinate (cm)")
    plt.ylabel("Y coordinate (cm)")
    plt.xlim(-210, 210)
    plt.ylim(-105, 105)
    plt.grid(True)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.show()


elmID, edep = ex3.get_hit_data()
edep = edep/ex3.sfc

# Plot without the conversion into spatial coordinates
mapped_matrix = map_channels_energies_to_matrix(elmID, edep)
plot_simple_energy_matrix(mapped_matrix)
# Plot with the conversion into spatial coordinates
coordinates = channel_to_spatial_coordinates(elmID)
plot_spatial_energy_matrix(coordinates, edep)

<div class="alert alert-info">
<strong>Exercise:</strong> 
What is the measured energy of the particle in event 5 of the electron sample? And what is the distribution of all measured energies in all events of the electron sample?</span>
</div>

In [ ]:
# The total energy for event 5 is rather trivial:
total_energy = np.sum(edep)
print(f'The total energy is {total_energy} GeV')

<a name='section_2_2'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 2.2: Cluster properties and moments</h3>

After having measured the energy deposited by all the hits in a cluster, it is important to characterize the cluster and determine its properties. We will use the moments of a distribution to do this. Remember that the clusters we are analyzing are basically 2D distributions!

<div class="alert alert-info">
<strong>Exercise:</strong> 
Using *numpy*, implement functions to calculate the mean (geometric center), width ($\sigma$), standardized skewness and standardized kurtosis for the PHENIX EMCal clusters.</span>
</div>

In [ ]:
def f_mean(data):
    return np.mean(data)


def f_width(data):
    return np.std(data, ddof=0)


def f_variance(data):
    return f_width(data)**2


def f_skewness(data):
    n = len(data)
    mean = f_mean(data)
    width = f_width(data)
    skewness = np.sum((data - mean) ** 3) / (n * (width ** 3))
    return skewness


def f_kurtosis(data):
    n = len(data)
    mean = f_mean(data)
    width = f_width(data)
    kurtosis = np.sum((data - mean) ** 4) / (n * (width ** 4))
    return kurtosis

<div class="alert alert-info">
<strong>Exercise:</strong> 
Visualize again some events from the electron sample and calculate the moments. Which moments look useful for identifying electrons, and why?</span>
</div>

In [ ]:
x = coordinates[:, 0]
y = coordinates[:, 1]

plot_spatial_energy_matrix(coordinates, edep)

print(x, y)
x_mean = f_mean(x)
y_mean = f_mean(y)
print(f'Mean: {x_mean},{y_mean}')
x_width = f_width(x)
y_width = f_width(y)
print(f'Width: {x_width},{y_width}')
x_skewness = f_skewness(x)
y_skewness = f_skewness(y)
print(f'Skewness: {x_skewness},{y_skewness}')
x_kurtosis = f_kurtosis(x)
y_kurtosis = f_kurtosis(y)
print(f'Kurtosis: {x_kurtosis},{y_kurtosis}')

<div class="alert alert-info">
<strong>Exercise:</strong> 
(Optional) Implement functions to calculate skewness and kurtosis without "standardization" and compute them for few events. Why do we usually use the standardized versions?</span>
</div>

In [ ]:
def f_skewness_raw(data):
    n = len(data)
    mean = f_mean(data)
    width = f_width(data)
    skewness = np.sum((data - mean) ** 3) / n
    return skewness


def f_kurtosis_raw(data):
    n = len(data)
    mean = f_mean(data)
    width = f_width(data)
    kurtosis = np.sum((data - mean) ** 4) / n
    return kurtosis


print(f'Skewness: {x_skewness},{y_skewness}')
x_skewness_raw = f_skewness_raw(x)
y_skewness_raw = f_skewness_raw(y)
print(f'Raw skewness: {x_skewness},{y_skewness}')
print(f'Kurtosis: {x_kurtosis},{y_kurtosis}')
x_kurtosis_raw = f_kurtosis_raw(x)
y_kurtosis_raw = f_kurtosis_raw(y)
print(f'Raw kurtosis: {x_kurtosis_raw},{y_kurtosis_raw}')


<a name='section_3_0'></a>
<hr style="height: 1px;">


## <h1 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Section 3: Calorimetry and clustering</h1>

<a name='section_3_1'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 3.1: An homemade K-means clustering algorithm</h3>

<div class="alert alert-info">
<strong>Exercise:</strong> 
Visualize the event 0 from the electron sample and the event 6 from the dielectron sample and then compute the relevant moments. Do they still provide a good description for multi-particle cases? Why?</span>
</div>

In [ ]:
# Event 0, electron
elmID, edep = ex3.get_hit_data()
edep = edep/ex3.sfc

coordinates = channel_to_spatial_coordinates(elmID)

x = coordinates[:, 0]
y = coordinates[:, 1]

plot_spatial_energy_matrix(coordinates, edep)

print(x, y)
x_mean = f_mean(x)
y_mean = f_mean(y)
print(f'Mean: {x_mean},{y_mean}')
x_width = f_width(x)
y_width = f_width(y)
print(f'Width: {x_width},{y_width}')
x_skewness = f_skewness(x)
y_skewness = f_skewness(y)
print(f'Skewness: {x_skewness},{y_skewness}')
x_kurtosis = f_kurtosis(x)
y_kurtosis = f_kurtosis(y)
print(f'Kurtosis: {x_kurtosis},{y_kurtosis}')

# Event 6, dielectron
elmID, edep = ex3.get_hit_data()
edep = edep/ex3.sfc

coordinates = channel_to_spatial_coordinates(elmID)

x = coordinates[:, 0]
y = coordinates[:, 1]

plot_spatial_energy_matrix(coordinates, edep)

print(x, y)
x_mean = f_mean(x)
y_mean = f_mean(y)
print(f'Mean: {x_mean},{y_mean}')
x_width = f_width(x)
y_width = f_width(y)
print(f'Width: {x_width},{y_width}')
x_skewness = f_skewness(x)
y_skewness = f_skewness(y)
print(f'Skewness: {x_skewness},{y_skewness}')
x_kurtosis = f_kurtosis(x)
y_kurtosis = f_kurtosis(y)
print(f'Kurtosis: {x_kurtosis},{y_kurtosis}')

For cases with particle gun decays, we can perform clustering to group the hits associated with different secondary particles produced in the decay.

To test the clustering algorithms and evaluate if they are implemented correctly, you can use the following method to randomly generate a number of clusters with a given number of hits.

```
import ex3
points = ex3.generate_2d_points()
# ex3.generate_2d_points(num_clusters=X, points_per_cluster=Y, spread=Z, random_seed=42)
```

<div class="alert alert-info">
<strong>Exercise:</strong> 
Generate some points with the default settings (without passing arguments) and also with some custom settings and visualize the generated datasets.</span>
</div>

In [ ]:
def generate_2d_points(num_clusters, points_per_cluster, spread, random_seed=42):
    np.random.seed(random_seed)
    data = []

    for i in range(num_clusters):
        center = np.random.uniform([-120, -60], [120, 60])
        cluster_points = center + np.random.randn(points_per_cluster, 2) * spread
        data.append(cluster_points)

    data = np.vstack(data)
    return data


def get_constant_array(array, constant=1):
    return np.full(array.shape[0], constant)


i=0
for n in [2, 4, 6]:
    for p in [10, 20, 30]:
        for s in [1., 3., 5.]:
            i+=1
            coordinates = generate_2d_points(num_clusters=n, points_per_cluster=p, spread=s, random_seed=i)
            energies = get_constant_array(array=coordinates, constant=1)
            plot_spatial_energy_matrix(coordinates=coordinates, energies=energies, title=f'Clusters: {n}, Points per clusters: {p}, Spread: {s}')

<div class="alert alert-info" role="alert">

<h3>🧩 Exercise: K-Means Clustering</h3>

<hr>

<h4>What is Cluster Analysis?</h4>

<p>
Cluster analysis is the process of <strong>grouping together similar points into clusters</strong>.
A <em>point</em> can have 2, 3, or even hundreds of dimensions — meaning it’s a vector in some space.
</p>

<p>
One practical example is <strong>epidemiological clustering</strong>:
you might have 2D points representing the <strong>longitude and latitude</strong> of locations where birds carrying different strains of avian flu were found.
By clustering these points, you can gain insight into which regions correspond to each strain.
</p>

<hr>

<h4>Distances Between Points</h4>

<p>
To cluster data, we must measure <strong>how close or far</strong> points are from each other.
The <strong>Euclidean distance</strong> is the most common measure.
</p>

<p>For two 2D points:</p>
\[
d(p, q) = \sqrt{(p_x - q_x)^2 + (p_y - q_y)^2}
\]

<p>For two 3D points:</p>
\[
d(p, q) = \sqrt{(p_x - q_x)^2 + (p_y - q_y)^2 + (p_z - q_z)^2}
\]

<p>In general, for two \(n\)-dimensional points
\(p = (p_1, p_2, \dots, p_n)\) and \(q = (q_1, q_2, \dots, q_n)\):</p>
\[
d(p, q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + \cdots + (p_n - q_n)^2}
\]

<p><strong>Example:</strong></p>
\[
p = (0.1, 0.2, 0.3, 0.4), \quad q = (0.0, 0.2, 0.3, 0.2)
\]
\[
d(p, q) = \sqrt{(0.1-0.0)^2 + (0.2-0.2)^2 + (0.3-0.3)^2 + (0.4-0.2)^2} = 0.7071\ldots
\]

<hr>

<h4>Cluster Centroids</h4>

<p>
The <strong>centroid</strong> of a cluster is its <em>center of mass</em> — the <strong>average position</strong> of all points in that cluster.
Even though it’s the mean of all cluster points, the centroid does <em>not</em> need to be one of those points.
</p>

<p>To compute a centroid:</p>
\[
\text{centroid} = \frac{1}{N} \sum_{i=1}^{N} p_i
\]

<p>Vector operations:</p>
\[
p + q = (p_1 + q_1,\, p_2 + q_2,\, \dots,\, p_n + q_n)
\]
\[
\frac{p}{a} = \left(\frac{p_1}{a},\, \frac{p_2}{a},\, \dots,\, \frac{p_n}{a}\right)
\]

<hr>

<h4>The K-Means Algorithm</h4>

<p>
The <strong>idea</strong> behind K-Means is simple:
each point belongs to the cluster whose centroid (mean) it is <strong>closest</strong> to.
Because centroids depend on which points are assigned to them, and assignments depend on centroids, we solve this <em>chicken-and-egg</em> problem iteratively.
</p>

<h5>Step 1. Pick \(k\) Centroids</h5>

<p>
We start by choosing \(k\), the number of clusters we want.
Each centroid \(m_j\) is an \(n\)-dimensional point:
</p>
\[
m_j = (m_{j,1}, m_{j,2}, \dots, m_{j,n})
\]
<p>
We randomly select \(k\) points from the dataset as our <strong>initial centroids</strong>
(the <em>Forgy method</em>).
</p>

<h5>Step 2. Partition the Dataset</h5>

<p>
Assign each point to the <strong>nearest centroid</strong> by Euclidean distance.
If a point is equally close to multiple centroids, break ties arbitrarily.
This produces \(k\) sets \(S_1, S_2, \dots, S_k\),
where each \(S_j\) contains all points closest to centroid \(m_j\).
</p>

<h5>Step 3. Recompute the Means</h5>

<p>For each cluster \(S_j\), recompute its centroid:</p>
\[
m_j = \frac{1}{|S_j|} \sum_{q \in S_j} q
\]
<p>
Since centroids have changed, some points may now be closer to a different centroid —
so we repeat the assignment step.
</p>

<h5>Step 4. Repeat Until Convergence</h5>

<ul>
  <li>The centroids stop moving, or</li>
  <li>The assignments no longer change.</li>
</ul>

<p>
Sometimes convergence takes many iterations, so we often set a
<strong>maximum iteration limit</strong> to prevent infinite loops.
</p>

<hr>

<h4>Summary of the K-Means Algorithm</h4>

<ol>
  <li>Choose \(k\) and initialize centroids randomly.</li>
  <li>Assign each point to the nearest centroid.</li>
  <li>Recompute each centroid as the mean of its assigned points.</li>
  <li>Repeat steps 2–3 until centroids stop changing or max iterations reached.</li>
</ol>

<hr>

<p><strong>✅ Key Idea:</strong>  
K-Means minimizes the <em>within-cluster sum of squared errors</em> (inertia) —  
the total squared distance between points and their assigned centroids.</p>

</div>


In [ ]:
import numpy as np
import random

class KMeans:
    """
    Minimal NumPy-only K-Means with functionized steps.
    No sklearn, no typing — pure Python version.

    Parameters
    ----------
    n_clusters : int
        Number of clusters (K).
    init : {"k-means++", "random"}
        Initialization scheme.
    max_iter : int
        Maximum iterations.
    tol : float
        Convergence tolerance on centroid shift (Euclidean norm).
    random_state : int or None
        Seed for reproducibility.
    """

    def __init__(self, n_clusters, init="k-means++", max_iter=300, tol=1e-4, random_state=None):
        self.n_clusters = int(n_clusters)
        self.init = init
        self.max_iter = int(max_iter)
        self.tol = float(tol)
        self.random_state = random_state

        # Attributes set after fitting
        self.cluster_centers_ = None
        self.labels_ = None
        self.inertia_ = None
        self.n_iter_ = None

    # ---------- core primitives ----------

    def _euclidean_squared(self, a, b):
        """
        Squared Euclidean distances between rows of a (n_samples, d)
        and rows of b (n_centroids, d). Returns (n_samples, n_centroids).
        """
        a2 = np.sum(a * a, axis=1, keepdims=True)
        b2 = np.sum(b * b, axis=1)
        ab = a @ b.T
        return a2 - 2.0 * ab + b2
    
    
    def _kmeanspp_init(self, X, rng):
        """
        k-means++ initialization. Returns (K, d) array of initial centroids.
        """
        n_samples, n_features = X.shape
        centroids = np.empty((self.n_clusters, n_features), dtype=X.dtype)

        # 1) choose first centroid uniformly
        idx0 = rng.integers(0, n_samples)
        centroids[0] = X[idx0]

        # 2) choose remaining with probability proportional to D^2
        closest_dist_sq = self._euclidean_squared(X, centroids[0:1]).ravel()
        for i in range(1, self.n_clusters):
            probs = closest_dist_sq / closest_dist_sq.sum()
            idx = rng.choice(n_samples, p=probs)
            centroids[i] = X[idx]
            d2 = self._euclidean_squared(X, centroids[i:i+1]).ravel()
            closest_dist_sq = np.minimum(closest_dist_sq, d2)
        return centroids

    # ---------- step-by-step methods ----------
    #rng = np.random.default_rng(42)
    def _initialize_centroids(self, X, rng=None):
        """Step 2: Initialize centroids randomly."""
        n_samples = X.shape[0]
        # Set Python random seed if random_state was provided
        if self.random_state is not None:
            random.seed(self.random_state)
        # Choose unique random indices for centroids
        indices = random.sample(range(n_samples), self.n_clusters)
        centroids = X[indices].copy()
        return centroids

    def _assign(self, X, centroids):
        """Step 3: Assignment — return (labels, d2)."""
        d2 = self._euclidean_squared(X, centroids)
        labels = np.argmin(d2, axis=1)
        return labels, d2

    def _update(self, X, labels, rng):
        """Step 4: Update — compute new centroids, handle empty clusters by re-seeding."""
        K, d = self.n_clusters, X.shape[1]
        new_centroids = np.empty((K, d), dtype=float)
        for j in range(K):
            mask = (labels == j)
            if np.any(mask):
                new_centroids[j] = X[mask].mean(axis=0)
            else:
                # reinitialize empty cluster to a random point
                new_centroids[j] = X[rng.integers(0, X.shape[0])]
        return new_centroids

    def _converged(self, old_centroids, new_centroids, old_labels, new_labels):
        """Step 5: Convergence check — label stability OR centroid shift < tol."""
        if old_labels is not None and np.array_equal(old_labels, new_labels):
            return True
        shift = float(np.sqrt(np.sum((new_centroids - old_centroids) ** 2)))
        return shift < self.tol

    def _compute_inertia(self, X, labels, centroids):
        """Step 6: Inertia (within-cluster SSE)."""
        return float(np.sum((X - centroids[labels]) ** 2))

    # ---------- public API ----------

    def fit(self, X):
        """
        Run K-Means on X.
        """
        X = np.asarray(X, dtype=float)
        rng = np.random.default_rng(self.random_state)

        centroids = self._initialize_centroids(X, rng)
        labels = None

        for it in range(1, self.max_iter + 1):
            new_labels, _ = self._assign(X, centroids)
            new_centroids = self._update(X, new_labels, rng)

            if self._converged(centroids, new_centroids, labels, new_labels):
                centroids = new_centroids
                labels = new_labels
                break

            centroids = new_centroids
            labels = new_labels

        # Final metrics
        inertia = self._compute_inertia(X, labels, centroids)

        # Set attributes
        self.cluster_centers_ = centroids
        self.labels_ = labels
        self.inertia_ = inertia
        self.n_iter_ = it
        return self

    def predict(self, X):
        """
        Assign cluster indices for new samples using fitted centroids.
        """
        if self.cluster_centers_ is None:
            raise RuntimeError("Model is not fitted. Call .fit(X) first.")
        X = np.asarray(X, dtype=float)
        labels, _ = self._assign(X, self.cluster_centers_)
        return labels

    def fit_predict(self, X):
        """
        Convenience: fit the model and return labels for X.
        """
        self.fit(X)
        return self.labels_


<div class="alert alert-info">
<strong>Unit test 1:</strong> 
use the cell below to evalue inertia and iterations K-Means you have implemented</span>
</div>

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans as SKKMeans
from sklearn.metrics import adjusted_rand_score

# ---- Load Iris dataset ----
iris = load_iris()
X = iris.data
y_true = iris.target

# ---- Your implementation ----
my_km = KMeans(n_clusters=3, init="k-means++", max_iter=300, tol=1e-4, random_state=34)
my_labels = my_km.fit_predict(X)

print("=== Your KMeans ===")
print(f"Iterations: {my_km.n_iter_}")
print(f"Inertia: {my_km.inertia_:.4f}")
print(f"ARI vs true labels: {adjusted_rand_score(y_true, my_labels):.4f}")

# ---- scikit-learn implementation ----
sk_km = SKKMeans(n_clusters=3, init="k-means++", n_init=1, max_iter=300, tol=1e-4,
                 algorithm="lloyd", random_state=42)
sk_labels = sk_km.fit_predict(X)

print("\n=== scikit-learn KMeans ===")
print(f"Iterations: {sk_km.n_iter_}")
print(f"Inertia: {sk_km.inertia_:.4f}")
print(f"ARI vs true labels: {adjusted_rand_score(y_true, sk_labels):.4f}")


<div class="alert alert-info">
<strong>Unit test 2:</strong> 
Import the external unit test, check if home-brewed K-Means has passed all the test</span>
</div>

In [ ]:
%run -i unit_test.py

<a name='section_3_2'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 3.2: Finding optimal number of centroids</h3>

<div class="alert alert-info">
<strong>Exercise:</strong> 
Implement by yourself an elbow method for your K-means algorithm and test it on few events to evaluate its performance.</span>
</div>

In [ ]:
def evaluate_method(data, score_method, title_label, min_k=1, max_k=10):
    score_values = []

    for k in range(min_k, max_k + 1):
        centroids, labels = k_means(data, k)
        score = score_method(data, centroids, labels)
        energies = get_constant_array(data)
        plot_spatial_energy_matrix_complex(data, energies, centroids=centroids, labels=labels, title=f'{title_label} - Clusters: {k} - Score: {score}')
        score_values.append(score)

    plt.figure(figsize=(10, 10))
    plt.plot(range(min_k, max_k + 1), score_values, marker='o', linestyle='--')
    plt.title(f"{title_label} method for the optimal K")
    plt.xlabel("Number of clusters (K)")
    plt.ylabel("Score")
    plt.show()


def wcss_score(data, centroids, labels):
    wcss = 0
    k = len(centroids)
    for i in range(k):
        cluster_points = data[labels == i]
        centroid = centroids[i]
        squared_distances = np.sum((cluster_points - centroid) ** 2, axis=1)
        wcss += np.sum(squared_distances)
    return wcss


for n in [2, 4, 6]:
    print('###############################################################################################')
    print(f'Generating {n} clusters...')
    coordinates = generate_2d_points(num_clusters=n, points_per_cluster=20, spread=4., random_seed=999)
    evaluate_method(data=coordinates, score_method=wcss_score, title_label='Elbow method', min_k=2, max_k=10)

<div class="alert alert-info">
<strong>Exercise:</strong> 
Implement by yourself a silhouette method for your K-means algorithm and test it on few events to evaluate its performance.</span>

In [ ]:
def silhouette_score(data, centroids, labels):
    total_score = 0
    n_samples = data.shape[0]
    k = len(centroids)

    for i in range(n_samples):
        point = data[i]
        point_label = labels[i]

        # Calculate the intra-cluster distance a_i
        same_cluster_points = data[labels == point_label]
        intra_cluster_distances = np.sum((same_cluster_points - point) ** 2, axis=1)
        a_i = np.sum(intra_cluster_distances) / len(same_cluster_points)

        # Calculate the nearest cluster distance b_i
        # Calculate the nearest cluster distance b_i
        b_i = float('inf')
        for j in range(k):
            if j == point_label:
                continue  # Skip the same cluster
            other_cluster_points = data[labels == j]
            inter_cluster_distances = np.sum((other_cluster_points - point) ** 2, axis=1)
            nearest_cluster_distance = np.sum(inter_cluster_distances) / len(other_cluster_points)
            if nearest_cluster_distance < b_i:
                b_i = nearest_cluster_distance

        # Calcualte the silhouette hit score s_i
        #if np.isinf(a_i) or np.isinf(b_i):
        #    s_i = 0
        if max(a_i, b_i) == 0:
            s_i = 0
        else:
            s_i = (b_i - a_i) / max(a_i, b_i)
        total_score += s_i

    # Return the average silhouette score S of the event
    return total_score / n_samples


for n in [2, 4, 6]:
    print('###############################################################################################')
    print(f'Generating {n} clusters...')
    coordinates = generate_2d_points(num_clusters=n, points_per_cluster=20, spread=4., random_seed=999)
    evaluate_method(data=coordinates, score_method=silhouette_score, title_label='Silhouette method', min_k=2, max_k=10)

<div class="alert alert-info">
<strong>Exercise:</strong> 
comparison of computational complexity for elbow and silhouette</span>


<h4>Computational Complexity of <em>Elbow</em> vs <em>Silhouette</em> Methods</h4>

<p>Let:</p>

<ul>
  <li>\( n \) — number of points</li>
  <li>\( d \) — dimensionality</li>
  <li>\( k \) — clusters</li>
  <li>\( I \) — K-Means iterations until convergence</li>
  <li>\( K_{\max} \) — maximum number of clusters you scan in your model selection loop</li>
</ul>

<hr>

<h5>A) Elbow Method (WCSS / Inertia)</h5>

<ol>
  <li>Run K-Means once per \(k\):  
      Cost \( \approx \mathcal{O}(n \cdot k \cdot I \cdot d) \) (standard Lloyd’s algorithm).</li>
  <li>Compute WCSS / inertia: obtained during the same pass; extra \( \mathcal{O}(n \cdot d) \).</li>
</ol>

<p>
Per \(k\):  
\[
\boxed{\mathcal{O}(n \cdot k \cdot I \cdot d)}
\]
Scan \(k = 2..K_{\max}\):  
\[
\boxed{\mathcal{O}(K_{\max} \cdot n \cdot k \cdot I \cdot d)}
\]
Memory: \( \mathcal{O}(n \cdot d) \)
</p>

<hr>

<h5>B) Silhouette Method</h5>

<p>
You still need to run K-Means per \(k\):  
\( \mathcal{O}(n \cdot k \cdot I \cdot d) \)
</p>

<p>Then compute silhouette:</p>

<ul>
  <li><strong>Naïve per-point:</strong> for each point, distances to all others  
      \( \Rightarrow \mathcal{O}(n^2 \cdot d) \)</li>
  <li><strong>Precompute distance matrix (per \(k\)):</strong>  
      \( \mathcal{O}(n^2 \cdot d) \) time and \( \mathcal{O}(n^2) \) memory,  
      then reuse to get \(a_i, b_i\) in \( \mathcal{O}(n) \).</li>
</ul>

<p>
Per \(k\) (naïve or precomputed):  
\[
\boxed{\mathcal{O}(n \cdot k \cdot I \cdot d) \;+\; \mathcal{O}(n^2 \cdot d)}
\]
Scan \(k = 2..K_{\max}\):  
\[
\boxed{\mathcal{O}\!\left(K_{\max} \cdot n \cdot k \cdot I \cdot d \;+\; K_{\max} \cdot n^2 \cdot d\right)}
\]
Memory (with precompute): \( \mathcal{O}(n^2) \)
</p>

</div>


<div class="alert alert-info">
<strong>Exercise:</strong> 
For particle physics, what tricks you can come up to reduce the computation complexity of number and coordination for the centroids</span>

In [ ]:
def seed_search(channels, energies, min_energy_threshold=5.0, padding_size=1, n_rows=72, n_columns=36):
    # Step 1: Map energies to the 2D energy matrix
    energy_matrix = map_channels_energies_to_matrix(channels, energies, n_rows, n_columns)

    # Step 2: Apply minimum energy threshold to identify potential seed candidates
    # Creating a binary mask where energy values are greater than the threshold
    energy_mask = energy_matrix > min_energy_threshold

    # Step 3: Get the coordinates of the potential seeds
    seed_coordinates = np.argwhere(energy_mask)

    # Step 4: Sort seeds by their energy values (ascending)
    seed_energies = energy_matrix[energy_mask]
    sorted_seeds = sorted(zip(seed_energies, seed_coordinates), key=lambda x: x[0])

    # Step 5: Define a padding region and check for neighboring cells with higher energy
    valid_seeds = []
    for energy, (x, y) in sorted_seeds:
        # Create a region around the current seed (including padding)
        x_min = max(0, x - padding_size)
        x_max = min(n_rows, x + padding_size + 1)
        y_min = max(0, y - padding_size)
        y_max = min(n_columns, y + padding_size + 1)

        # Extract the neighboring region (within bounds)
        region = energy_matrix[x_min:x_max, y_min:y_max]

        # If any neighboring cell has energy greater than the current seed, discard it
        if np.any(region > energy):
            continue

        # Otherwise, keep this seed
        channel_id = y * n_columns + x
        valid_seeds.append(channel_id)

    if not valid_seeds:
        return None

    return valid_seeds

# This will be evaluated later :)

<a name='section_3_2'></a>
<hr style="height: 1px;">


## <h3 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #FFA500">Problem 3.3: Resolution of EM Calorimeter</h3>

<div class="alert alert-info">
<strong>Exercise:</strong> 

#  **B4d Calorimeter Geometry Overview**

B4d defines a **longitudinal sampling electromagnetic calorimeter**:
a stack of alternating **absorber** and **active (gap)** layers, just like B4b, but with **sensitive detectors** and **scorers** automatically attached.

---

##  **Geometric parameters (default values)**

| Quantity                     | Symbol          | Default Value         | Description                                |
| ---------------------------- | --------------- | --------------------- | ------------------------------------------ |
| Number of layers             | `nofLayers`     | 10                    | Number of absorber + gap pairs             |
| Absorber thickness           | `absoThickness` | 10 mm                 | Thickness of lead absorber per layer       |
| Gap (active layer) thickness | `gapThickness`  | 5 mm                  | Thickness of liquid argon gap per layer    |
| Calorimeter transverse size  | `calorSizeXY`   | 10 cm × 10 cm         | Square cross-section perpendicular to beam |
| World volume size            | `1.2×` larger   | 12 cm × 12 cm × 18 cm | Vacuum surrounding the calorimeter         |

---

##  **Material composition**

| Volume          | Material            | Notes                                             |
| --------------- | ------------------- | ------------------------------------------------- |
| **Absorber**    | `G4_Pb` (lead)      | Dense converter for e⁻/γ showers                  |
| **Gap**         | `liquidArgon`       | Active readout medium (collects deposited energy) |
| **World**       | `Galactic` (vacuum) | Empty container for geometry                      |
| **Calorimeter** | (composite)         | Logical container holding 10 layers               |

---

##  **Structure (Z-axis stacking)**

Each layer consists of:

```
|<--- Layer (15 mm) ----------------------------->|
|                                                 |
|   [ Absorber (Pb) | 10 mm ]  +  [ Gap (LAr) | 5 mm ] |
```

Stack 10 such layers along **+z** (the beam direction), producing a total depth of **150 mm**.

### Schematic:

```
  e⁻ beam →
            |##########|-----|##########|-----|##########|-----|
            |  Pb (10) | LAr |  Pb (10) | LAr |  Pb (10) | LAr |
            |##########|-----|##########|-----|##########|-----|
                    <----------- repeated 10 times ----------->
```

---

##  **World hierarchy**

| Level | Volume name   | Parent        | Description            |
| ----- | ------------- | ------------- | ---------------------- |
| 0     | `World`       | —             | Vacuum box             |
| 1     | `Calorimeter` | `World`       | Container of layers    |
| 2     | `Layer`       | `Calorimeter` | Replicated 10× along z |
| 3     | `Abso`        | `Layer`       | Lead absorber          |
| 3     | `Gap`         | `Layer`       | Liquid argon gap       |

---

#  **Physical meaning**

* **Absorber (Pb):** Converts incident e⁻ or γ into a cascade of secondary particles via bremsstrahlung and pair production.
* **Gap (LAr):** Collects the energy deposits of the shower particles; this “visible energy” is your calorimeter signal.
* Each layer combination corresponds to one **sampling unit**.
* Total thickness (~150 mm) gives about **15 radiation lengths**, enough to contain a several-GeV electromagnetic shower.
</span>

In [ ]:
import subprocess, shutil, pathlib, textwrap, os
import uproot
import numpy as np
import matplotlib.pyplot as plt

# ---- CONFIG ----
EXE = "exampleB4d.py"     # <- use the B4d example
N_EVENTS = 100           # bump stats for smoother σ/E
energies_GeV = [1, 2, 5, 10, 20, 50]

# ---- RUN ONE ENERGY AND RENAME OUTPUT ----
def run_one_energy(E):
    mac = pathlib.Path(f"run_{E}GeV.mac")
    mac.write_text(textwrap.dedent(f"""
        /run/initialize
        /gun/particle e-
        /gun/energy {E} GeV
        /run/printProgress 1000
        /run/beamOn {N_EVENTS}
    """).strip())

    # ensure no leftover file from previous run
    if pathlib.Path("B4.root").exists():
        os.remove("B4.root")

    # run the example in batch mode with this macro
    subprocess.run(["python", EXE, "-m", str(mac)], check=True)

    # the example writes B4.root; rename to per-energy file
    src = pathlib.Path("B4.root")
    if not src.exists():
        raise FileNotFoundError("Expected B4.root not found. Did the run finish?")
    dst = pathlib.Path(f"B4_{E}GeV.root")
    shutil.move(str(src), str(dst))
    return str(dst)

# run all energies
roots = [run_one_energy(E) for E in energies_GeV]
print("Wrote:", roots)

# ---- LOAD Edep IN GAP (VISIBLE ENERGY) ----
def load_edep_gap(root_path):
    with uproot.open(root_path) as f:
        t = f["B4"]                    # TTree name
        Egap = t["Egap"].array(library="np")  # MeV
    return Egap

means, sigmas, resolutions = [], [], []
for E, rfile in zip(energies_GeV, roots):
    Egap = load_edep_gap(rfile)       # MeV per event
    mu   = Egap.mean()                # MeV
    sig  = Egap.std(ddof=1)           # MeV
    R    = sig / mu                   # dimensionless
    means.append(mu)
    sigmas.append(sig)
    resolutions.append(R)

means  = np.array(means)
sigmas = np.array(sigmas)
R      = np.array(resolutions)
E      = np.array(energies_GeV, dtype=float)

# ---- FIT R(E) = sqrt(a^2/E + b^2) VIA LINEARIZATION ----
x = 1.0 / E
y = R**2
A = np.vstack([x, np.ones_like(x)]).T
m, c = np.linalg.lstsq(A, y, rcond=None)[0]  # y ≈ m x + c
a = float(np.sqrt(max(m, 0.0)))  # stochastic term (√GeV units)
b = float(np.sqrt(max(c, 0.0)))  # constant term

print(f"a = {a*100:.1f}%·√GeV")
print(f"b = {b*100:.2f}%")

# ---- PLOT ----
E_dense = np.linspace(E.min(), E.max(), 200)
R_fit = np.sqrt( (a*a)/E_dense + b*b )

plt.plot(E, R, "o", label="data (σ/⟨E⟩)")
plt.plot(E_dense, R_fit, "-", label=f"fit: a={a*100:.1f}%√GeV, b={b*100:.2f}%")
plt.xlabel("Beam energy E (GeV)")
plt.ylabel("Energy resolution σ/⟨E⟩")
plt.grid(True)
plt.legend()
plt.show()


Hey, congrats you made all the way through!  
It’s okay if you don’t understand everything yet — join the help session for some useful ideas.  
Enjoy the process, keep curious, you will be fine!